# 配置

使用前请确保 HM3D 的标签和配置对应的 `tar` 文件已经解压完成，但如果场景对应的 `tar` 文件过大，则无需解压，直接运行

如果需要改变存储的格式，请随意修改 `save_sample()`

In [ ]:
settings = {
    "width": 256,
    "height": 256,
    'T_per_sample': 100, # 每个场景中，随机产生多少个位置
    'rotation_step': 360.0, # 在每个位置上，产生该值除 360 个样本，相邻两个样本间摄像机朝向相差该值；该值为 360 表示每个位置采一个样本

    # 数据集位置
    "dataset": "../hm3d_train/",
    "dataset_config": "../hm3d_train/hm3d_annotated_train_basis.scene_dataset_config.json", # Configuration of the dataset
    "classes_json": "../rgbx/minival/classes.json",


    # 采样结果的储存位置
    "rgb_path": "../../../data/rgbx/train/rgb",
    "depth_path": "../../../data/rgbx/train/depth",
    "semantic_path": "../../../data/rgbx/train/label",

    "sensor_height": 1.5,  # 传感器的高度，单位：米
    "enable_physics": False, 
}

In [ ]:
import numpy as np
def save_sample(rgb_obs: np.ndarray, depth_obs: np.ndarray, semantic_obs: np.ndarray, rgb_path: str, depth_path: str, semantic_path: str):
    rgb_img = Image.fromarray(rgb_obs, mode="RGBA")
    rgb_img.save(rgb_path)

    np.save(depth_path, depth_obs)
    np.save(semantic_path, semantic_obs)

配置完成后运行剩下的全部代码，即可在 `settings['rgb_path'/'depth_path'/'semantic_path']` 下找到采样后的 RGB/D/Semantic 图像。

# 主体部分

In [ ]:
from utils import make_cfg, make_samples, display_sample

## 在单独场景中采样

In [ ]:
from models.perception.utils import ClassManager
import habitat_sim
def sample(scene_name, sim, T: int, rgb_path: str, depth_path: str, semantic_path: str, display: bool=False):
    import random
    import os
    cm = ClassManager.from_sim(sim, settings['classes_json'])
    paths = {'color': rgb_path, 'depth': depth_path, 'semantic': semantic_path}
    for t in range(T):
        pos = None
        while pos is None or not sim.pathfinder.is_navigable(pos):
            pos = sim.pathfinder.get_random_navigable_point()
        rot = np.array(random.random() * 2 * np.pi)
        orientation = np.quaternion(np.cos(rot / 2), 0, -1 * np.sin(rot / 2), 0)
        # orientation = np.quaternion(1, 0, 0, 0)
        state = habitat_sim.AgentState(position=pos, rotation=orientation, sensor_states=sim.get_agent(0).get_state().sensor_states)
        sim.get_agent(0).set_state(state=state, reset_sensors=False)

        for k in range(int(360 / settings['rotation_step'])):
            obs = sim.step('turn_right')
            if display:
                (obs['color_sensor'], cm.instance_to_global(obs['semantic_sensor']), obs['depth_sensor'])
            prefix = scene_name + "_" + f"{pos}".replace(' ','').replace('[', '').replace(']', '') + "_" + f"{k}"
            save_sample(
                obs['color_sensor'], obs['depth_sensor'], cm.instance_to_global(obs['semantic_sensor']), 
                os.path.join(rgb_path, prefix + ".png"),
                os.path.join(depth_path, prefix + ".npy"),
                os.path.join(semantic_path, prefix + ".npy"),
                )

In [ ]:
def make_samples_for_scene(sim: habitat_sim.Simulator,scene_dir:str, scene_name:str=None):
    sample(scene_name, sim, settings['T_per_sample'], settings['rgb_path'], settings['depth_path'], settings['semantic_path'])

## 在所有场景中采样

In [ ]:
make_samples(settings, make_samples_for_scene)